# Importando módulos 

In [ ]:
import obspy
from obspy.taup import TauPyModel

from multiprocessing import Pool
from obspy import read,UTCDateTime,Trace,read_inventory,read_events
from obspy.io.sac.sactrace import SACTrace
from obspy.imaging.beachball import beachball,beach
from obspy.clients.fdsn import Client
from obspy.signal.trigger import recursive_sta_lta,delayed_sta_lta,trigger_onset
from obspy.signal.util import next_pow_2

import os
import glob
import numpy as np
from collections import defaultdict
import pandas as pd
from scipy import signal
import subprocess
from sklearn import preprocessing
import geopy.distance

import pywt
from matplotlib import mlab

from tslearn.preprocessing import TimeSeriesScalerMeanVariance,TimeSeriesResampler
from tslearn.clustering import TimeSeriesKMeans

#para plotar as figuras
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.transforms import offset_copy
import matplotlib.ticker as ticker
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import ListedColormap
import matplotlib.dates as mdates
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import matplotlib.cm as cm
from matplotlib.dates import YearLocator, MonthLocator, DayLocator, HourLocator, MinuteLocator, SecondLocator, DateFormatter
from matplotlib.ticker import MultipleLocator, FormatStrFormatter,FixedLocator,StrMethodFormatter
import matplotlib.patches as mpatches
import matplotlib.colors as mcolors
import matplotlib.gridspec as gridspec

from datetime import datetime,timedelta,date
from tqdm import tqdm
from kneed import KneeLocator

from shapely.geometry.polygon import LinearRing
from matplotlib.patches import Rectangle

import cartopy.io.shapereader as shpreader
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.ticker import LongitudeFormatter,LatitudeFormatter
import requests
import csv
import xml.etree.ElementTree as ET

# Inputs e Outputs

In [ ]:
FOLDER_OUTPUT = '/home/sysop/dados_posdoc/GLIDER_PETROBRAS/OUTPUT/'
MSEED_INPUT = "/home/sysop/dados_posdoc/GLIDER_PETROBRAS/DATA/"
METADATA_FILE = '/home/sysop/dados_posdoc/GLIDER_PETROBRAS/data_glider_information_csv/metadados_glider_acustico_pmpas-bs.csv'
QUAKEXML_FOLDER = '/home/sysop/dados_posdoc/GLIDER_PETROBRAS/OUTPUT/EVENTS_FINDER/'

# Functions

In [ ]:
def psd_cal(st,fmax=50.,winlen_HF=4.,winlen_LF=60.,overlap=0.5):
    
    '''
    fmax: Maximum frequency for plot
    winlen_HF: Window length for high-frequency spectrogram (in seconds)
    winlen_LF: Window length for long-period spectrogram (in seconds)
    overlap: Percentage of overlap (between 0 and 1) for spectrogram computation, if kind=='spec'
    '''
    
    for tr in st.slide(window_length=winlen_LF,step=winlen_LF/2,include_partial_windows=False):   
        
        # The computation of the LF spectrograms with long time windows or even CWT
        # can be REALLY slow, thus, decimate it to anything larger 2.5 Hz
        st_LF = tr.copy()        
        
        # ----------------------
        
        # Decimate it to anything larger 2.5 Hz
        st_LF.filter('lowpass', freq=0.95, corners=16)
        
        # For LF, 2 Hz is enough
        st_LF.interpolate(sampling_rate=10,method='nearest')
        
        # ----------------------

        st_HF = tr.copy()
        
        # ----------------------

        winlen_LF == int(winlen_LF * st_LF.stats.sampling_rate)
        winlen_HF == int(winlen_HF * st_HF.stats.sampling_rate)

        # ----------------------

        if st_LF.stats.sampling_rate > 10:
            dec_fac = int(st_LF.stats.sampling_rate / 10)
            st_LF.decimate(dec_fac)

        # ----------------------

        for st in [st_HF, st_LF]:
            st.detrend()
            st.filter('highpass', freq=1. / winlen_LF)

        # ----------------------
        fmin_LF = 2/winlen_LF
        fmax_LF = 1
        
        fmin_HF = 0.8
        fmax_HF = fmax
        
        fft_dic = {'data_FFT_HF':[],'freq_FFT_HF':[],'data_FFT_LF':[],'freq_FFT_LF':[],'starttime':[],'endtime':[]}

        p_HF, f_HF = mlab.psd(preprocessing.normalize([st_HF.data])[0],Fs=st_HF.stats.sampling_rate,noverlap=int(winlen_LF*overlap))
        bol_HF = np.array((f_HF >= fmin_HF * 0.9) & (f_HF <= fmax_HF))

        p_LF, f_LF = mlab.psd(preprocessing.normalize([st_LF.data])[0],Fs=st_LF.stats.sampling_rate,noverlap=int(winlen_LF*overlap))
        bol_LF = np.array((f_LF >= fmin_LF * 0.9) & (f_LF <= fmax_LF))

        fft_dic['data_FFT_HF'] = 10 * np.log10(p_HF[bol_HF])
        fft_dic['freq_FFT_HF'] = f_HF[bol_HF]
        
        fft_dic['data_FFT_LF'] = 10 * np.log10(p_LF[bol_LF])
        fft_dic['freq_FFT_LF'] = f_LF[bol_LF]
        
        fft_dic['starttime'] =  tr.stats.starttime.datetime
        fft_dic['endtime'] =  tr.stats.endtime.datetime

        if not pd.isna(fft_dic) and not pd.isnull(fft_dic):
            return fft_dic

# Lista com informações dos Gliders:

In [ ]:
GL_lst = sorted(list(set([i.split('/')[8] for i in glob.glob(MSEED_INPUT+'*/*/*/*/'+'*GL.*')])))

In [ ]:
GL_lst

# Criando a identidade frequencial dos equipamentos

In [ ]:
for seletected_STA in tqdm(GL_lst,desc='Processing GL data',total=len(GL_lst),position=0,leave=True):
    if os.path.exists(FOLDER_OUTPUT+'FIGURAS/PSD/GL/'+seletected_STA+'/df_psd_LF_'+seletected_STA+'_KMeans.feather'):
        pass
    else:
            
        files_mseed_ev = sorted(glob.glob(MSEED_INPUT+'*/*/*/*/'+'*GL.'+seletected_STA+'*'+'..HHH.D.*'))
        stream_mseed = [read(file) for file in files_mseed_ev]
        
        flat_list = [item for sublist in stream_mseed for item in sublist]
    
        # ========================================================================================================= #
        
        # --------------------------
        # Calculando os PSDs
        
        df_results = []
        with Pool(processes=20) as p:
            max_ = len(flat_list)
            with tqdm(total=max_,position=0,leave=False) as pbar:
                for result in p.imap_unordered(psd_cal,flat_list):
                    df_results.append(result)           
                    pbar.update()
    
        # --------------------------
        # Remove NaN and None values
        df_results_clean = [x for x in df_results if not pd.isna(x)]
    
        # --------------------------
        # Criando o Dataframe com os PSDs calculados
        fft_data_df = pd.DataFrame(df_results_clean)
        # Convert 'date' to datetime object, if not already}
        fft_data_df['starttime'] = pd.to_datetime(fft_data_df['starttime'])
        fft_data_df['endtime'] = pd.to_datetime(fft_data_df['endtime'])
            
        # --------------------------
        # Agrupando o Dataframe com os PSDs calculados a cada 1 hora
        df_grouped = fft_data_df.groupby(pd.Grouper(key='starttime', freq='1h')).mean()
        # Drop rows with any NaN values
        df_grouped = df_grouped.dropna()
        
        if not df_grouped.shape[0] > 10:
            pass
        else:
        
            ################################
            ##### CREATING THE FIGURE ######
            ################################
            
            # Locator: X & Y
            majorX = MultipleLocator(5)
            majorY = MultipleLocator(20)
            
            minorX = MultipleLocator(1)
            minorY = MultipleLocator(10)
            
            fig_data = plt.figure(figsize=(20,10))
            
            # [left bottom width height]
            w_total = 1
            h_base = 0.13
            w_base = 0.08
            plot_ratio=0.3
            
            w_LF = w_total * plot_ratio
            w_HF = w_total - w_LF

            alpha = 0.75
            lw = 0.1
            
            ax_psd_LF = fig_data.add_axes([w_base, h_base, w_LF, 0.7],label='PSD LF')
            ax_psd_HF = fig_data.add_axes([w_base+plot_ratio, h_base,w_HF,0.7],label='PSD HF',sharey=ax_psd_LF)
            #(left, bottom, width, height)

            # Normalizing dates
            date_nums = mdates.date2num(df_grouped['endtime'])
            norm = plt.Normalize(vmin=min(date_nums), vmax=max(date_nums))  # Normaliza para o range das datas
            cmap = plt.get_cmap("cividis")
            
            # Plot PSD computed via both methods
            for i in df_grouped.iterrows():
                year_quarter = i[1]['endtime']
                color = cmap(norm(mdates.date2num(year_quarter)))
                ax_psd_LF.plot(i[1]['freq_FFT_LF'], i[1]['data_FFT_LF'], color=color, lw=lw, alpha=alpha)
                ax_psd_HF.plot(i[1]['freq_FFT_HF'], i[1]['data_FFT_HF'], color=color, lw=lw, alpha=alpha)
            
            ax_psd_LF.plot(df_grouped['freq_FFT_LF'].mean(),df_grouped['data_FFT_LF'].mean(), color='k', lw=1)
            ax_psd_HF.plot(df_grouped['freq_FFT_HF'].mean(),df_grouped['data_FFT_HF'].mean(), color='k', lw=1)
            
            # Set limits for x-axes
            ax_psd_LF.set_xlim(0.01, 1)
            ax_psd_HF.set_xlim(1, 50)
        
            # Set labels for axes
            ax_psd_LF.set_ylabel('Spectral level (dB)')
            ax_psd_LF.set_xlabel('Frequency (Hz)')
        
            ax_psd_HF.text(0.9, 0.9,'Total = '+str(df_grouped.shape[0]),fontweight='bold', fontsize=12, ha='center',va='center',transform=ax_psd_HF.transAxes)
            ax_psd_HF.text(0.1, 0.9,seletected_STA,fontweight='bold', fontsize=15, ha='right',va='center',transform=ax_psd_HF.transAxes)
            
            # Set ticks and labels on both sides of the y-axis
            ax_psd_LF.yaxis.set_tick_params(labelleft=True,labelright=False,left=True, right=True)  # Show labels on the right side
            ax_psd_LF.spines['top'].set_linewidth(2)
            ax_psd_LF.spines['right'].set_linewidth(2)
            ax_psd_LF.spines['bottom'].set_linewidth(2)
            ax_psd_LF.spines['left'].set_linewidth(2)
            ax_psd_LF.xaxis.set_tick_params(width=2)
            ax_psd_LF.yaxis.set_tick_params(width=2)
        
            ax_psd_LF.xaxis.set_major_locator(MultipleLocator(0.3))
            ax_psd_LF.xaxis.set_minor_locator(MultipleLocator(0.1))
            ax_psd_LF.yaxis.set_major_locator(majorY)
            ax_psd_LF.yaxis.set_minor_locator(minorY)
        
            # Set ticks and labels on both sides of the y-axis
            ax_psd_HF.yaxis.set_tick_params(labelleft=False,labelright=True,left=True, right=True)  # Show labels on the right side
            ax_psd_HF.spines['top'].set_linewidth(2)
            ax_psd_HF.spines['right'].set_linewidth(2)
            ax_psd_HF.spines['bottom'].set_linewidth(2)
            ax_psd_HF.spines['left'].set_linewidth(2)
            ax_psd_HF.xaxis.set_tick_params(width=2)
            ax_psd_HF.yaxis.set_tick_params(width=2)
            
            ax_psd_HF.xaxis.set_major_locator(majorX)
            ax_psd_HF.xaxis.set_minor_locator(minorX)
            ax_psd_HF.yaxis.set_major_locator(majorY)
            ax_psd_HF.yaxis.set_minor_locator(minorY)
                
            # Create a scalar mappable for the colorbar
            sm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin=mdates.date2num(df_grouped['endtime'].values[0]), vmax=mdates.date2num(df_grouped['endtime'].values[-1])))
            
            # Create the colorbar
            cbar = fig_data.colorbar(sm, ax=[ax_psd_LF, ax_psd_HF], orientation='vertical')
            cbar.set_label('Days of the year')
            
            # Set the ticks and tick labels for the colorbar
            ticks = np.linspace(cbar.vmin, cbar.vmax, 6)  # Gera ticks uniformemente espaçados
            cbar.set_ticks(ticks)
            cbar.ax.yaxis.set_major_formatter(mdates.DateFormatter('%d/%m/%Y'))
            
            os.makedirs(FOLDER_OUTPUT+'FIGURAS/PSD/GL/'+seletected_STA+'/',exist_ok=True)
            fig_data.savefig(FOLDER_OUTPUT+'FIGURAS/PSD/GL/'+seletected_STA+'/GL_'+seletected_STA+'_Data.png',pad_inches=0.02,dpi=200)
            
            # ========================================================================================================= #
            
            # ------------------------------------
            # Adjusting arrays to KMeans procedure
            X_train = TimeSeriesResampler(sz=len(df_grouped['data_FFT_HF'].values[0])).fit_transform(df_grouped['data_FFT_HF'].values)
        
            # ------------------------------------
            # Elbow KMeans estimation
            elbow_data = []
            for n_clusters in tqdm(range(1,10,1),total=len(range(1,10,1)),desc='Elbow Method',position=0,leave=False):
            
                km = TimeSeriesKMeans(n_clusters=n_clusters, verbose=False, random_state=0,n_jobs=20)
                y_pred = km.fit_predict(X_train)
                elbow_data.append((n_clusters, km.inertia_))
        
            k_range = []
            inertias = []
            for elb in elbow_data:
                k_range.append(elb[0])
                inertias.append(elb[1])

            # ------------------------------------    
            # Elbow KMeans estimation
            kn = KneeLocator(k_range, inertias,S=2, curve='convex', direction='decreasing')
            elbow_point = kn.knee+1
            
            # ------------------------------------
            # Elbow KMeans plot
            
            fig_elb, ax_elb = plt.subplots(figsize=(10,5))
            
            ax_elb.annotate(str(elbow_point), xy=(elbow_point,inertias[k_range.index(elbow_point)]), xytext=(elbow_point+(elbow_point*0.5),inertias[k_range.index(elbow_point)]+(inertias[k_range.index(elbow_point)]*0.5)),arrowprops=dict(arrowstyle='<-',linewidth=0.25,color='black'))
            ax_elb.scatter(elbow_point,inertias[k_range.index(elbow_point)],color='k', marker='X',s=100)
            ax_elb.plot(k_range,inertias,color='grey', marker='o', linestyle='dashed',linewidth=2, markersize=5,zorder=-1)
            ax_elb.text(0.85, 0.9,seletected_STA,fontweight='bold', fontsize=25, ha='right',va='center',transform=ax_elb.transAxes)
            
            # Set ticks and labels on both sides of the y-axis
            ax_elb.set_xlim(0,10)
            ax_elb.set_ylabel('Inertias')
            ax_elb.set_xlabel('Values of K')
            fig_elb.savefig(FOLDER_OUTPUT+'FIGURAS/PSD/GL/'+seletected_STA+'/GL_'+seletected_STA+'_Elbow_plot.png',pad_inches=0.02,dpi=200)

            # ------------------------------------
            # Euclidean k-means
           
            n_clu = elbow_point
            km = TimeSeriesKMeans(n_clusters=n_clu, verbose=False, random_state=0)
            y_pred = km.fit_predict(X_train)
            
            df_grouped['k-means'] = y_pred
            
            ################################
            ##### CREATING THE FIGURE ######
            ################################
            
            fig = plt.figure(figsize=(20,10))
            gs = gridspec.GridSpec(3, n_clu)
            
            ax_psd_HF = fig.add_subplot(gs[:2, :])
               
            # Plot PSD computed via both methods
            for i in df_grouped.iterrows():
                year_quarter = i[1]['endtime']
                color = cmap(norm(mdates.date2num(year_quarter)))
                ax_psd_HF.plot(i[1]['freq_FFT_HF'], i[1]['data_FFT_HF'], color=color, lw=lw, alpha=alpha)
            
            ax_psd_HF.plot(df_grouped['freq_FFT_HF'].mean(),df_grouped['data_FFT_HF'].mean(), color='k', lw=2)
            ax_psd_HF.text(0.9, 0.9,'Total = '+str(df_grouped.shape[0]),fontweight='bold', fontsize=12, ha='center',va='center',transform=ax_psd_HF.transAxes)
            ax_psd_HF.text(0.1, 0.9,seletected_STA,fontweight='bold', fontsize=15, ha='right',va='center',transform=ax_psd_HF.transAxes)
            
            ax_psd_HF.set_ylabel('Spectral level (dB)')
            
            # Set limits for x-axes
            ax_psd_HF.set_xlim(1, 50)
            ax_psd_HF.set_ylim(-120,-20)
            
            # Set ticks and labels on both sides of the y-axis
            ax_psd_HF.yaxis.set_tick_params(labelright=True,left=True, right=True)  # Show labels on the right side
            ax_psd_HF.spines['top'].set_linewidth(2)
            ax_psd_HF.spines['right'].set_linewidth(2)
            ax_psd_HF.spines['bottom'].set_linewidth(2)
            ax_psd_HF.spines['left'].set_linewidth(2)
            ax_psd_HF.xaxis.set_tick_params(width=2)
            ax_psd_HF.yaxis.set_tick_params(width=2)
            
            ax_psd_HF.xaxis.set_major_locator(majorX)
            ax_psd_HF.xaxis.set_minor_locator(minorX)
            ax_psd_HF.yaxis.set_major_locator(majorY)
            ax_psd_HF.yaxis.set_minor_locator(minorY)
            
            # =======================================================================================================================================
            
            # Get unique k-means
            unique_k_means = sorted(df_grouped['k-means'].unique())
            
            for k in unique_k_means:
                ax = fig.add_subplot(gs[2, k])
            
                df_k = df_grouped[df_grouped['k-means'] == k]
            
                # Plot PSD computed via both methods
                for i in df_k.iterrows():
                    year_quarter = i[1]['endtime']
                    color = cmap(norm(mdates.date2num(year_quarter)))
                    ax.plot(i[1]['freq_FFT_HF'], i[1]['data_FFT_HF'], color=color, lw=lw, alpha=alpha)
                ax.plot(df_k['freq_FFT_HF'].mean(),df_k['data_FFT_HF'].mean(), color='k', lw=2)
                ax.text(0.5, 0.8,'Cluster %d' % (k + 1)+'\n(n='+str(df_k.shape[0])+')',fontweight='bold', fontsize=12, ha='center',transform=ax.transAxes)
            
                ax.set_xlabel('Frequency (Hz)')
                ax.set_ylim(-120,-20)
                ax.set_xlim(1, 50)
            
                ax.spines['top'].set_linewidth(2)
                ax.spines['right'].set_linewidth(2)
                ax.spines['bottom'].set_linewidth(2)
                ax.spines['left'].set_linewidth(2)
                ax.xaxis.set_tick_params(width=2)
                ax.yaxis.set_tick_params(width=2)  
            
                ax.xaxis.set_major_locator(majorX)
                ax.xaxis.set_minor_locator(minorX)
                ax.yaxis.set_major_locator(majorY)
                ax.yaxis.set_minor_locator(minorY)
                ax.yaxis.set_tick_params(labelright=False,labelleft=False,left=True, right=True)  # Show labels on the right side
            
                if k == unique_k_means[0]:
                    ax.yaxis.set_tick_params(labelright=False,labelleft=True,left=True, right=True)  # Show labels on the right side
                    ax.set_ylabel('Spectral level (dB)')
            
                if k == unique_k_means[-1]:
                    ax.yaxis.set_tick_params(labelright=True,labelleft=False,left=True, right=True)  # Show labels on the right side
            
            # Create a color bar
            # Create a scalar mappable for the colorbar
            sm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin=mdates.date2num(df_grouped['endtime'].values[0]), vmax=mdates.date2num(df_grouped['endtime'].values[-1])))
            
            # Create the colorbar
            cbar = fig.colorbar(sm,ax=ax_psd_HF,orientation='vertical')
            cbar.set_label('Days of the year')
            
            # Set the ticks and tick labels for the colorbar
            ticks = np.linspace(cbar.vmin, cbar.vmax, 6)  # Gera ticks uniformemente espaçados
            cbar.set_ticks(ticks)
            cbar.ax.yaxis.set_major_formatter(mdates.DateFormatter('%d/%m/%Y'))

            fig.savefig(FOLDER_OUTPUT+'FIGURAS/PSD/GL/'+seletected_STA+'/GL_'+seletected_STA+'_KMeans.png',pad_inches=0.02,dpi=200)
        
            # =======================================================================================================================================
            # Saving 
            # =======================================================================================================================================
            df_grouped.to_feather(FOLDER_OUTPUT+'FIGURAS/PSD/GL/'+seletected_STA+'/df_psd_GL_'+seletected_STA+'_KMeans.feather') 
        
            # ----------------------------------------------------------------------------------------------------------
            # Close the figure after savig 
            # ----------------------------------------------------------------------------------------------------------                             
                                            
            plt.close('all')